In [1]:
import requests
import tkinter as tk
from tkinter import messagebox
import webbrowser

CHEAPSHARK = "https://www.cheapshark.com/api/1.0"
DEALS_URL = CHEAPSHARK + "/deals"
STORES_URL = CHEAPSHARK + "/stores"
FX_URL = "https://api.exchangerate-api.com/v4/latest/USD"
REDIRECT = "https://www.cheapshark.com/redirect?dealID={deal_id}"
